# Reddit Scraper

Pushshift: [RedditScraper](https://melaniewalsh.github.io/Intro-Cultural-Analytics/04-Data-Collection/14-Reddit-Data.html)

In [ ]:
import pandas as pd
pd.set_option('max_colwidth',500)
# pd.set_option('max_columns', 50)

In [5]:
from pmaw import PushshiftAPI
api= PushshiftAPI()

In [6]:
# start date = 2022-01-01
# end date = 2022-07-01
import datetime as dt



In [38]:
def data_prep_posts(subreddit, start_time, end_time):
    # if(len(filters)==0):
    #     filters = ['id', 'author','domain','created_utc','url','title','num_comments','score']
    posts = list(api.search_submissions(
        subreddit=subreddit,
        since = start_time,
        until= end_time
    ))
    return pd.DataFrame(posts)

In [70]:
def data_prep_comments(term, start_time, end_time, filters):
    if (len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'body', 'permalink', 'subreddit','score']
                   #We set by default some usefull columns

    comments = list(api.search_comments(
        q=term,                 #Subreddit we want to audit
        since=start_time,       #Start date
        until=end_time,        #End date

    ))
        # limit=limit))           #Max number of comments
    return pd.DataFrame(comments) #Return dataframe for analysis

In [78]:

subreddit= "bitcoin"
start = int(dt.datetime(2022,12,1).timestamp())
end = int(dt.datetime(2022,12,5).timestamp())

In [79]:
start

1669852800

In [81]:
term="bitcoin"
filters=[]
df_p = data_prep_comments(term,start,end, filters)

Not all PushShift shards are active. Query results may be incomplete.


In [77]:
df_p

""


In [ ]:

df_p['datetime'] = df_p['created_utc'].map(
        lambda t: dt.datetime.fromtimestamp(t))
df_p = df_p.drop('created_utc', axis=1)
df_p = df_p.sort_values(by='datetime')

In [ ]:
df_p

In [94]:
import datetime as datetime
import pandas as pd
subreddit = "Hannaford"     #Subreddit we are auditing
start_time = int(datetime.datetime(2022, 11, 5).timestamp())
                                 #Starting date for our search
end_time = int(datetime.datetime(2022, 11, 30).timestamp())
                                 #Ending date for our search
# filters = ['selftext']

limit = 10000                   #Elelemts we want to recieve

# Example of general use
from pmaw import PushshiftAPI
api = PushshiftAPI()

# The `search_comments` and `search_submissions` methods return generator objects
gen = api.search_submissions(subreddit=subreddit, since=start_time, until=end_time, size=limit)
results = list(gen)
results_df = pd.DataFrame(results)
results_df.head(10)

Not all PushShift shards are active. Query results may be incomplete.


""


In [ ]:

results_df = results_df.rename(columns={'num_comments': 'comms_num'})
results_df = results_df.rename(columns={'selftext': 'body'})
results_df['timestamp'] = pd.to_datetime(results_df['created_utc'], unit='s')


In [93]:
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import requests
import time

start_date = dt.date(2023, 1, 1)
end_date = dt.date(2023, 2, 15)

start_date = dt.datetime.fromordinal(start_date.toordinal())
end_date = dt.datetime.fromordinal(end_date.toordinal())

api = PushshiftAPI()

start_epoch = int(start_date.timestamp())
end_epoch = int(end_date.timestamp())

submissions = api.search_submissions(subreddit='politics', q='biden', since=start_epoch,
                                     until=end_epoch, num_workers=20)

sub_df = pd.DataFrame(submissions)
print('---------------------------------------------------------------')
print(f'pmaw df size: {sub_df.shape}')
print(sub_df.head())

time.sleep(10)
# Pushsift api directily
api_query = 'https://api.pushshift.io/reddit/submission/search/?q=' + 'biden' \
            + '&since=' + str(start_epoch) \
            + '&until=' + str(end_epoch) \
            + '&subreddit=' + 'politics' \
            + '&limit=' + str(100)

r = requests.get(api_query)
json = r.json()
df_pushshift = pd.DataFrame(json['data'])

print('---------------------------------------------------------------')
print(f'Pushshift direct df size: {df_pushshift.shape}')
print(df_pushshift.head())
print('---------------------------------------------------------------')

Not all PushShift shards are active. Query results may be incomplete.


---------------------------------------------------------------
pmaw df size: (0, 0)
Empty DataFrame
Columns: []
Index: []


KeyError: 'data'